# Calculate the distance to stores in London by car from a given location

**Table of contents**:
* [Introduction](#Introduction)
* [Calculate travel time and travel distance](#Calculate-travel-time-and-travel-distance)
* [Pretty-print-the-results](#Pretty-print-the-results)
* [Sort by distance](#Sort-by-distance)


## Introduction

[CalculateRouteMatrix](https://docs.aws.amazon.com/location/latest/APIReference/API_CalculateRouteMatrix.html) provides a travel cost matrix, also known as the origin-destination (OD) cost matrix representing the **travel time** and **travel distance from one or multiple origins to one or multiple destinations**. It can be used to:

* Calculate and sort by distance the nearest shops to a customer
* Find the closest vehicle to a location (e.g. ambulance to an accident, taxi to a customer, etc.).
* Etc.

![calculate_route_matrix](./images/calculate_route_matrix.png)

**Prerequisites:**
* AWS SDK for Python ([Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html))
* [Python-dotenv](https://pypi.org/project/python-dotenv/) reads key-value pairs from a .env file and can set them as environment variables.
* [pandas](https://pandas.pydata.org/) for data manipulation

In [ ]:
import boto3
from dotenv import load_dotenv
import os
import pandas as pd

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Create a client for Amazon Location service
amazon_location_client = boto3.client(
    "location",
    aws_access_key_id = os.getenv("AWS_ACCESS_KEY"),
    aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name = os.getenv("AWS_REGION"),
)

## Calculate travel time and travel distance

In [ ]:
# Calculate distance from a single location to multiple destinations
Departures = [
    [-0.1417089267, 51.5787616672]
]
Destinations = [
    [-0.1254974319, 51.50852485],
    [-0.1342243716, 51.52619485],
    [-0.1231958457, 51.53036755],
    [-0.08229997944, 51.51856002],
    [-0.1726801947, 51.51721979],
    [-0.1259365099, 51.53253756],
    [-0.1425202953, 51.49654998],
    [-0.1117242765, 51.50333369]
]

response = amazon_location_client.calculate_route_matrix(
    CalculatorName = 'AWS-Esri-Demo',
    TravelMode = 'Car',
    DeparturePositions = Departures,
    DestinationPositions = Destinations,
    DistanceUnit = 'Kilometers'
)
response

Learn more about [CalculateRouteMatrix response](https://docs.aws.amazon.com/location/latest/APIReference/API_CalculateRouteMatrix.html#API_CalculateRouteMatrix_ResponseSyntax)

## Pretty print the results

In [47]:
import json
print(f'RouteMatrix = %s' % (json.dumps(response['RouteMatrix'], indent=2)))
print(f'SnappedDeparturePositions = %s' % (json.dumps(response['SnappedDeparturePositions'], indent=2)))
print(f'SnappedDestinationPositions = %s' % (json.dumps(response['SnappedDestinationPositions'], indent=2)))
print(f'Summary = %s' % (json.dumps(response['Summary'], indent=2)))

RouteMatrix = [
  [
    {
      "Distance": 9.757,
      "DurationSeconds": 1497
    },
    {
      "Distance": 7.567,
      "DurationSeconds": 1030
    },
    {
      "Distance": 6.823,
      "DurationSeconds": 929
    },
    {
      "Distance": 9.059,
      "DurationSeconds": 1422
    },
    {
      "Distance": 10.065,
      "DurationSeconds": 1446
    },
    {
      "Distance": 7.212,
      "DurationSeconds": 1016
    },
    {
      "Distance": 12.44,
      "DurationSeconds": 1792
    },
    {
      "Distance": 11.098,
      "DurationSeconds": 1641
    }
  ]
]
SnappedDeparturePositions = [
  [
    -0.141674,
    51.578288
  ]
]
SnappedDestinationPositions = [
  [
    -0.125497,
    51.508525
  ],
  [
    -0.134224,
    51.526195
  ],
  [
    -0.123196,
    51.530368
  ],
  [
    -0.082079,
    51.519485
  ],
  [
    -0.17268,
    51.51722
  ],
  [
    -0.125937,
    51.532538
  ],
  [
    -0.14252,
    51.49655
  ],
  [
    -0.111724,
    51.503334
  ]
]
Summary = {
  "DataSource": 

## Sort by distance

In [48]:
df = pd.DataFrame({
    'Origin': ', '.join(map(str, Departures[0])),
    'Destination': Destinations
})
new_df = pd.DataFrame(response['RouteMatrix'][0])
extended_df = pd.concat([df, new_df], axis=1)
extended_df.sort_values('Distance')

,Origin,Destination,Distance,DurationSeconds
2,"-0.1417089267, 51.5787616672","[-0.1231958457, 51.53036755]",6.823,929
5,"-0.1417089267, 51.5787616672","[-0.1259365099, 51.53253756]",7.212,1016
1,"-0.1417089267, 51.5787616672","[-0.1342243716, 51.52619485]",7.567,1030
3,"-0.1417089267, 51.5787616672","[-0.08229997944, 51.51856002]",9.059,1422
0,"-0.1417089267, 51.5787616672","[-0.1254974319, 51.50852485]",9.757,1497
4,"-0.1417089267, 51.5787616672","[-0.1726801947, 51.51721979]",10.065,1446
7,"-0.1417089267, 51.5787616672","[-0.1117242765, 51.50333369]",11.098,1641
6,"-0.1417089267, 51.5787616672","[-0.1425202953, 51.49654998]",12.440,1792
